https://adventofcode.com/2021/day/9

In [1]:
with open("q9_input.txt", "r") as fp:
    file_lines = fp.readlines()

In [2]:
grid = [[int(i) for i in l.strip()] for l in file_lines]

In [3]:
n_rows = len(grid)
n_cols = len(grid[0])

In [4]:
def check_neighbor(i: int, j: int) -> bool:
    """Check if the index is the local min compared with neighbors."""
    val = grid[i][j]
    for direction in [(1, 0), (0, 1), (-1, 0), (0, -1)]:
        new_i, new_j = i + direction[0], j + direction[1]
        if 0 <= new_i < n_rows and 0 <= new_j < n_cols:
            neighbor_val = grid[new_i][new_j]
            if neighbor_val <= val:
                return False
    return True

In [5]:
local_mins = []

for i in range(n_rows):
    for j in range(n_cols):
        if check_neighbor(i, j):
            local_mins.append((i, j))

In [6]:
ans_1 = sum(grid[i][j] for i, j in local_mins) + len(local_mins)
print(ans_1)

530


In [7]:
def check_neighbor_with_seen(i: int, j: int, seen: set) -> bool:
    """Check if the index is the local min compared with neighbors."""
    val = grid[i][j]
    for direction in [(1, 0), (0, 1), (-1, 0), (0, -1)]:
        new_i, new_j = i + direction[0], j + direction[1]
        if 0 <= new_i < n_rows and 0 <= new_j < n_cols and (new_i, new_j) not in seen:
            neighbor_val = grid[new_i][new_j]
            # ERRRRR, why would this condition be different for part 2!?
            # got a wrong answer first, suspected this is the only reason
            # if equal it is still counting as basin?!
            if neighbor_val < val:
                return False
    return True

In [8]:
%%time
from collections import deque

basin_size = [0 for _ in range(len(local_mins))]
past_sets = []
for k, local_min in enumerate(local_mins):
    # print("-->")
    q = deque([local_min])
    seen = set([local_min])
    while q:
        i, j = q.popleft()
        seen.add((i, j))
        # print(seen, q, grid[i][j], i, j)
        for direction in [(1, 0), (0, 1), (-1, 0), (0, -1)]:
            new_i, new_j = i + direction[0], j + direction[1]
            if (
                0 <= new_i < n_rows
                and 0 <= new_j < n_cols
                and (new_i, new_j) not in seen
                and check_neighbor_with_seen(new_i, new_j, seen)
                # 9 doesn't count
                and grid[new_i][new_j] != 9
            ):
                q.append((new_i, new_j))
                seen.add((new_i, new_j))
    #     for i, j in sorted(seen):
    #         print(i, j, grid[i][j])
    basin_size[k] = len(seen)
    past_sets.append(seen)

CPU times: user 44.3 ms, sys: 2.05 ms, total: 46.3 ms
Wall time: 45.8 ms


In [9]:
# check if there are overlaps (shouldn't)
for s in past_sets:
    for s2 in past_sets:
        if s != s2 and s.intersection(s2):
            print(s, s2)

In [10]:
ans_2 = 1
for bs in sorted(basin_size, reverse=True)[:3]:
    ans_2 *= bs
print(ans_2)

1019494
